In [1]:
# !pip install packaging

In [2]:
# !pip install pandas dash

In [3]:
# !pip3 install httpx==0.20 dash plotly

In [4]:
# Import required packages
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output


In [5]:
# Read the airline data into pandas dataframe
spacex_df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df.shape    # 56, 7
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# # Pie Chart Creation
# fig = px.pie(data, values='Flights', names='DistanceGroup', title='Distance group proportion by flights')
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


## TASK 1: Add a Launch Site Drop-down Input Component

In [8]:
app = dash.Dash(__name__)

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                     ],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                             ),
                                html.Br(),
                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites or the entered site.
                                html.Div(dcc.Graph(id='success-pie-chart')),

                                html.Br(),

                                # TASK 3: Add a Range Slider to Select Payload
                                dcc.RangeSlider(id='payload-slider', 
                                                min=0, max=10000, step=1000, 
                                                marks={0: '0', 2500: '2500',5000: '5000',7500: '7500', 10000: '10000'}, 
                                                value=[min_payload, max_payload]),
                                html.Br(),
                                
                                # TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot.
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])




## TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

In [9]:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        counts = filtered_df['class'].value_counts().reset_index()
        counts.columns = ['class', 'count']
        fig = px.pie(counts, values='count', names = 'class', title = f'Success Launches at {entered_site}', color='class',
                 color_discrete_map={0: 'blue', 1: 'red'})
        return fig

## TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

In [10]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')])

def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, y='class', x='Payload Mass (kg)', 
                     title='Total success-payload-scatter-chart', 
                     color="Booster Version Category")
        return fig
    else:
        filtered2_df = filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(filtered2_df, y='class', x='Payload Mass (kg)', 
                     title = f'Success-payload-scatter-chart at {entered_site}',
                     color="Booster Version Category")
        return fig

In [11]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)